# Bayesian Optimization for LightGBM

In this notebook I use Bayesian global optimization with gaussian processes for finding optimal parameters. This optimization attempts to find the maximum value of an black box function in as few iterations as possible. In our case the black box function will be a function that I will write to optimize (maximize) the evaluation function (AUC) so that parameters get maximize AUC in training and validation, and expect to do good in the private. The final prediction will be rank average on 5 fold cross validation predictions.


## Notebook  Content
0. [Installing Bayesian global optimization library](#0) <br>    
1. [Loading the data](#1)
2. [Black box function to be optimized (LightGBM)](#2)
3. [Training LightGBM model](#3)
4. [Rank averaging](#4)

<a id="0"></a> <br>
## 0. Installing Bayesian Optimization library

Let's install the latest release from pip

In [1]:
!pip install bayesian-optimization

<a id="1"></a> <br>
## 1. Loading the data

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from scipy.stats import rankdata
import lightgbm as lgb
from sklearn import metrics
import gc
import warnings

pd.set_option('display.max_columns', 200)

/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

Distribution of target variable

In [6]:
target = 'target'
predictors = train_df.columns.values.tolist()[2:]

In this kernel I will be using **50% Stratified rows** as holdout rows for the validation-set to get optimal parameters. Later I will use 5 fold cross validation in the final model fit.

In [8]:
bayesian_tr_index, bayesian_val_index  = list(StratifiedKFold(n_splits=2, shuffle=True, random_state=1).split(train_df, train_df.target.values))[0]

These `bayesian_tr_index` and `bayesian_val_index` indexes will be used for the bayesian optimization as training and validation index of training dataset.

<a id="2"></a> <br>
## 2. Black box function to be optimized (LightGBM)

As data is loaded, let's create the black box function for LightGBM to find parameters.

In [9]:
def LGB_bayesian(
    num_leaves,  # int
    min_data_in_leaf,  # int
    learning_rate,
    min_sum_hessian_in_leaf,    # int  
    feature_fraction,
    lambda_l1,
    lambda_l2,
    min_gain_to_split,
    max_depth):
    
    # LightGBM expects next three parameters need to be integer. So we make them integer
    num_leaves = int(num_leaves)
    min_data_in_leaf = int(min_data_in_leaf)
    max_depth = int(max_depth)

    assert type(num_leaves) == int
    assert type(min_data_in_leaf) == int
    assert type(max_depth) == int

    param = {
        'num_leaves': num_leaves,
        'max_bin': 63,
        'min_data_in_leaf': min_data_in_leaf,
        'learning_rate': learning_rate,
        'min_sum_hessian_in_leaf': min_sum_hessian_in_leaf,
        'bagging_fraction': 1.0,
        'bagging_freq': 5,
        'feature_fraction': feature_fraction,
        'lambda_l1': lambda_l1,
        'lambda_l2': lambda_l2,
        'min_gain_to_split': min_gain_to_split,
        'max_depth': max_depth,
        'save_binary': True, 
        'seed': 1337,
        'feature_fraction_seed': 1337,
        'bagging_seed': 1337,
        'drop_seed': 1337,
        'data_random_seed': 1337,
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': 1,
        'metric': 'auc',
        'is_unbalance': True,
        'boost_from_average': False,   

    }    
    
    
    xg_train = lgb.Dataset(train_df.iloc[bayesian_tr_index][predictors].values,
                           label=train_df.iloc[bayesian_tr_index][target].values,
                           feature_name=predictors,
                           free_raw_data = False
                           )
    xg_valid = lgb.Dataset(train_df.iloc[bayesian_val_index][predictors].values,
                           label=train_df.iloc[bayesian_val_index][target].values,
                           feature_name=predictors,
                           free_raw_data = False
                           )   

    num_round = 5000
    clf = lgb.train(param, xg_train, num_round, valid_sets = [xg_valid], verbose_eval=250, early_stopping_rounds = 50)
    
    predictions = clf.predict(train_df.iloc[bayesian_val_index][predictors].values, num_iteration=clf.best_iteration)   
    
    score = metrics.roc_auc_score(train_df.iloc[bayesian_val_index][target].values, predictions)
    
    return score

The above `LGB_bayesian` function will act as black box function for Bayesian optimization. I already defined the the trainng and validation dataset for LightGBM inside the `LGB_bayesian` function. 

The `LGB_bayesian` function takes values for `num_leaves`, `min_data_in_leaf`, `learning_rate`, `min_sum_hessian_in_leaf`, `feature_fraction`, `lambda_l1`, `lambda_l2`, `min_gain_to_split`, `max_depth` from Bayesian optimization framework. Keep in mind that `num_leaves`, `min_data_in_leaf`, and `max_depth` should be integer for LightGBM. But Bayesian Optimization sends continous vales to function. So I force them to be integer. I am only going to find optimal parameter values of them. The reader may increase or decrease number of parameters to optimize.

Now I need to give bounds for these parameters, so that Bayesian optimization only search inside the bounds.

In [10]:
# Bounded region of parameter space
bounds_LGB = {
    'num_leaves': (5, 20), 
    'min_data_in_leaf': (5, 20),  
    'learning_rate': (0.01, 0.3),
    'min_sum_hessian_in_leaf': (0.00001, 0.01),    
    'feature_fraction': (0.05, 0.5),
    'lambda_l1': (0, 5.0), 
    'lambda_l2': (0, 5.0), 
    'min_gain_to_split': (0, 1.0),
    'max_depth':(3,15),
}

Let's put all of them in BayesianOptimization object

In [11]:
from bayes_opt import BayesianOptimization

In [12]:
LGB_BO = BayesianOptimization(LGB_bayesian, bounds_LGB, random_state=13)

Now, let's the the key space (parameters) we are going to optimize:

In [13]:
print(LGB_BO.space.keys)

['feature_fraction', 'lambda_l1', 'lambda_l2', 'learning_rate', 'max_depth', 'min_data_in_leaf', 'min_gain_to_split', 'min_sum_hessian_in_leaf', 'num_leaves']


I have created the BayesianOptimization object (`LGB_BO`), it will not work until I call maximize. Before calling it, I want to explain two parameters of BayesianOptimization object (`LGB_BO`) which we can pass to maximize:
- `init_points`: How many initial random runs of **random** exploration we want to perform. In our case `LGB_bayesian` will be called `n_iter` times.
- `n_iter`: How many runs of bayesian optimization we want to perform after number of `init_points` runs. 

Now, it's time to call the function from Bayesian optimization framework to maximize. I allow `LGB_BO` object to run for 5 `init_points` (exploration) and 5 `n_iter` (exploitation).

In [14]:
init_points = 5
n_iter = 5

In [15]:
print('-' * 130)

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    LGB_BO.maximize(init_points=init_points, n_iter=n_iter, acq='ucb', xi=0.0, alpha=1e-6)

----------------------------------------------------------------------------------------------------------------------------------
|   iter    |  target   | featur... | lambda_l1 | lambda_l2 | learni... | max_depth | min_da... | min_ga... | min_su... | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[178]	valid_0's auc: 0.878546
|  1        |  0.8785   |  0.4      |  1.188    |  4.121    |  0.2901   |  14.67    |  11.8     |  0.609    |  0.007758 |  14.62    |
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.84073
[500]	valid_0's auc: 0.866356
[750]	valid_0's auc: 0.877017
[1000]	valid_0's auc: 0.882954
[1250]	valid_0's auc: 0.886643
[1500]	valid_0's auc: 0.888876
[1750]	valid_0's auc: 0.890244
[2000]	valid_0's auc: 0.891098
[2250]	valid_0's auc: 0.891491
Ear

As the optimization is done, let's see what is the maximum value we have got.

In [16]:
LGB_BO.max['target']

0.8931128566288707

The validation AUC for parameters is 0.89 ! Let's see parameters is responsible for this score :)

In [17]:
LGB_BO.max['params']

{'feature_fraction': 0.0725311213806508,
 'lambda_l1': 4.778909939998174,
 'lambda_l2': 3.220122059503974,
 'learning_rate': 0.07663544326133201,
 'max_depth': 14.99504485809263,
 'min_data_in_leaf': 6.071757005556453,
 'min_gain_to_split': 0.4634735801572327,
 'min_sum_hessian_in_leaf': 0.007510919138894667,
 'num_leaves': 5.001626877554514}

Now we can use these parameters to our final model!

Wait, I want to show one more cool option from BayesianOptimization library. You can probe the `LGB_bayesian` function, if you have an idea of the optimal parameters or it you get **parameters from other kernel** like mine [mine](https://www.kaggle.com/fayzur/customer-transaction-prediction). I will copy and paste parameters from my other kernel here. You can probe as folowing:

In [18]:
# parameters from version 2 of
#https://www.kaggle.com/fayzur/customer-transaction-prediction?scriptVersionId=10522231

LGB_BO.probe(
    params={'feature_fraction': 0.1403, 
            'lambda_l1': 4.218, 
            'lambda_l2': 1.734, 
            'learning_rate': 0.07, 
            'max_depth': 14, 
            'min_data_in_leaf': 17, 
            'min_gain_to_split': 0.1501, 
            'min_sum_hessian_in_leaf': 0.000446, 
            'num_leaves': 6},
    lazy=True, # 
)

OK, by default these will be explored lazily (lazy=True), meaning these points will be evaluated only the next time you call maximize. Let's do a maximize call of `LGB_BO` object.

In [19]:
LGB_BO.maximize(init_points=0, n_iter=0) # remember no init_points or n_iter

|   iter    |  target   | featur... | lambda_l1 | lambda_l2 | learni... | max_depth | min_da... | min_ga... | min_su... | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.86243
[500]	valid_0's auc: 0.880738
[750]	valid_0's auc: 0.887841
[1000]	valid_0's auc: 0.890905
[1250]	valid_0's auc: 0.892214
[1500]	valid_0's auc: 0.892655
Early stopping, best iteration is:
[1466]	valid_0's auc: 0.892672
|  11       |  0.8927   |  0.1403   |  4.218    |  1.734    |  0.07     |  14.0     |  17.0     |  0.1501   |  0.000446 |  6.0      |


Finally, the list of all parameters probed and their corresponding target values is available via the property LGB_BO.res.

In [20]:
for i, res in enumerate(LGB_BO.res):
    print("Iteration {}: \n\t{}".format(i, res))

Iteration 0: 
	{'target': 0.8785456504868869, 'params': {'feature_fraction': 0.3999660847582191, 'lambda_l1': 1.1877061001745615, 'lambda_l2': 4.1213926633068425, 'learning_rate': 0.2900672674324699, 'max_depth': 14.67121336685872, 'min_data_in_leaf': 11.801738711259683, 'min_gain_to_split': 0.6090424627612779, 'min_sum_hessian_in_leaf': 0.007757509880902418, 'num_leaves': 14.624200171386038}}
Iteration 1: 
	{'target': 0.8915776403640969, 'params': {'feature_fraction': 0.3749082032826262, 'lambda_l1': 0.17518262050718658, 'lambda_l2': 1.492247354445897, 'learning_rate': 0.026968622645801674, 'max_depth': 13.284731311046386, 'min_data_in_leaf': 10.592810418122113, 'min_gain_to_split': 0.679847951578097, 'min_sum_hessian_in_leaf': 0.002570236693773035, 'num_leaves': 10.21371822728738}}
Iteration 2: 
	{'target': 0.8924701144136038, 'params': {'feature_fraction': 0.05423574653643624, 'lambda_l1': 1.7916689135248487, 'lambda_l2': 4.745470908391052, 'learning_rate': 0.07319071264818977, 'max

We have got a better validation score in the probe! As previously I ran `LGB_BO` only for 10 runs. In practice I increase it to arround 100.

In [21]:
LGB_BO.max['target']

0.8931128566288707

In [22]:
LGB_BO.max['params']

{'feature_fraction': 0.0725311213806508,
 'lambda_l1': 4.778909939998174,
 'lambda_l2': 3.220122059503974,
 'learning_rate': 0.07663544326133201,
 'max_depth': 14.99504485809263,
 'min_data_in_leaf': 6.071757005556453,
 'min_gain_to_split': 0.4634735801572327,
 'min_sum_hessian_in_leaf': 0.007510919138894667,
 'num_leaves': 5.001626877554514}

Let's build a model together use therse parameters ;)

<a id="3"></a> <br>
## 3. Training LightGBM model

In [23]:
param_lgb = {
        'num_leaves': int(LGB_BO.max['params']['num_leaves']), # remember to int here
        'max_bin': 63,
        'min_data_in_leaf': int(LGB_BO.max['params']['min_data_in_leaf']), # remember to int here
        'learning_rate': LGB_BO.max['params']['learning_rate'],
        'min_sum_hessian_in_leaf': LGB_BO.max['params']['min_sum_hessian_in_leaf'],
        'bagging_fraction': 1.0, 
        'bagging_freq': 5, 
        'feature_fraction': LGB_BO.max['params']['feature_fraction'],
        'lambda_l1': LGB_BO.max['params']['lambda_l1'],
        'lambda_l2': LGB_BO.max['params']['lambda_l2'],
        'min_gain_to_split': LGB_BO.max['params']['min_gain_to_split'],
        'max_depth': int(LGB_BO.max['params']['max_depth']), # remember to int here
        'save_binary': True,
        'seed': 1337,
        'feature_fraction_seed': 1337,
        'bagging_seed': 1337,
        'drop_seed': 1337,
        'data_random_seed': 1337,
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': 1,
        'metric': 'auc',
        'is_unbalance': True,
        'boost_from_average': False,
    }

As you see, I assined `LGB_BO`'s optimal parameters to the `param_lgb` dictionary and they will be used to train a model with 5 fold.

Number of Kfolds:

In [24]:
nfold = 5

In [25]:
gc.collect()

32

In [26]:
skf = StratifiedKFold(n_splits=nfold, shuffle=True, random_state=2019)

In [27]:
oof = np.zeros(len(train_df))
predictions = np.zeros((len(test_df),nfold))

i = 1
for train_index, valid_index in skf.split(train_df, train_df.target.values):
    print("\nfold {}".format(i))
    xg_train = lgb.Dataset(train_df.iloc[train_index][predictors].values,
                           label=train_df.iloc[train_index][target].values,
                           feature_name=predictors,
                           free_raw_data = False
                           )
    xg_valid = lgb.Dataset(train_df.iloc[valid_index][predictors].values,
                           label=train_df.iloc[valid_index][target].values,
                           feature_name=predictors,
                           free_raw_data = False
                           )   

    
    clf = lgb.train(param_lgb, xg_train, 5000, valid_sets = [xg_valid], verbose_eval=250, early_stopping_rounds = 50)
    oof[valid_index] = clf.predict(train_df.iloc[valid_index][predictors].values, num_iteration=clf.best_iteration) 
    
    predictions[:,i-1] += clf.predict(test_df[predictors], num_iteration=clf.best_iteration)
    i = i + 1

print("\n\nCV AUC: {:<0.3f}".format(metrics.roc_auc_score(train_df.target.values, oof)))


fold 1
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.863083
[500]	valid_0's auc: 0.882892
[750]	valid_0's auc: 0.890602
[1000]	valid_0's auc: 0.894356
[1250]	valid_0's auc: 0.896589
[1500]	valid_0's auc: 0.89754
Early stopping, best iteration is:
[1658]	valid_0's auc: 0.89802

fold 2
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.858944
[500]	valid_0's auc: 0.878526
[750]	valid_0's auc: 0.887018
[1000]	valid_0's auc: 0.891189
[1250]	valid_0's auc: 0.893594
[1500]	valid_0's auc: 0.894757
[1750]	valid_0's auc: 0.895592
[2000]	valid_0's auc: 0.896141
Early stopping, best iteration is:
[2009]	valid_0's auc: 0.89618

fold 3
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.865096
[500]	valid_0's auc: 0.884309
[750]	valid_0's auc: 0.891393
[1000]	valid_0's auc: 0.895089
[1250]	valid_0's auc: 0.897185
[1500]	valid_0's auc: 0.898671
[1750]	valid_0's auc: 0.899236
Early stopping, bes

So we got 0.90 AUC in 5 fold cross validation. And 5 fold prediction look like:

In [28]:
predictions

array([[0.47431024, 0.39227261, 0.44468045, 0.38681385, 0.42184034],
       [0.68469791, 0.6810922 , 0.70206808, 0.58120489, 0.66681126],
       [0.732413  , 0.72693412, 0.59743011, 0.6587553 , 0.74861721],
       ...,
       [0.0649998 , 0.03296316, 0.04015894, 0.04720749, 0.02271844],
       [0.40141865, 0.43569536, 0.4639441 , 0.46392935, 0.37243934],
       [0.44078843, 0.39342836, 0.42481913, 0.49840102, 0.42225307]])

If you are still reading, bare with me. I will not take much of your time. :D We are almost done. Let's do a rank averaging on 5 fold predictions.

<a id="4"></a> <br>
## 4. Rank averaging

In [29]:
print("Rank averaging on", nfold, "fold predictions")
rank_predictions = np.zeros((predictions.shape[0],1))
for i in range(nfold):
    rank_predictions[:, 0] = np.add(rank_predictions[:, 0], rankdata(predictions[:, i].reshape(-1,1))/rank_predictions.shape[0]) 

rank_predictions /= nfold

Rank averaging on 5 fold predictions
